In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/linear_regression_nnm/pytorch/default/1/linear_regression_scale.pkl
/kaggle/input/linear_regression_nnm/pytorch/default/1/linear_regression_nnm.pth


In [2]:
import torch

# Neural Network model linear regression
# model, loss function, optimizer function
class LinearRegressionNNM(torch.nn.Module) :
    def __init__(self, input_dim): # input : feature 의 열 갯수
        super(LinearRegressionNNM, self).__init__()
        # super(self).__init__()
        self.hidden_1 = torch.nn.Linear(input_dim, 64) # input_dim : feature 수 10개, output_dim : 출력 수 64 개
        self.hidden_2 = torch.nn.Linear(64, 32) # input_dim : 이전 layer output_dim 수 64개, output_dim : 출력 수 32 개
        self.hidden_3 = torch.nn.Linear(32, 16) # input_dim : 이전 layer output_dim 수 32, output_dim : 출력 수 16 개
        self.output = torch.nn.Linear(16, 1) # input_dim : 이전 layer output_dim 수 16, output_dim : 출력 수 1 개 Linear 니까
        self.relu = torch.nn.ReLU() # activation func ReLU 쓸거임. 

    def forward(self, x):
        x = self.relu(self.hidden_1(x)) # hidden layer 1 10,64
        x = self.relu(self.hidden_2(x)) # hidden layer 2 64,32
        x = self.relu(self.hidden_3(x)) # hidden layer 3 32,16
        out = self.output(x) # hidden layer 4 16,1 result 
        
        return out

In [3]:
loaded_model = LinearRegressionNNM(15)
loaded_model

LinearRegressionNNM(
  (hidden_1): Linear(in_features=15, out_features=64, bias=True)
  (hidden_2): Linear(in_features=64, out_features=32, bias=True)
  (hidden_3): Linear(in_features=32, out_features=16, bias=True)
  (output): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
)

In [4]:
loaded_state_dict = torch.load("/kaggle/input/linear_regression_nnm/pytorch/default/1/linear_regression_nnm.pth")

<ipython-input-4-04a68e217918>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_state_dict = torch.load("/kaggle/input/linear_regression_nnm/pytorch/default/1/linear_

In [5]:
loaded_model.load_state_dict(loaded_state_dict)
loaded_model.eval()

LinearRegressionNNM(
  (hidden_1): Linear(in_features=15, out_features=64, bias=True)
  (hidden_2): Linear(in_features=64, out_features=32, bias=True)
  (hidden_3): Linear(in_features=32, out_features=16, bias=True)
  (output): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
)

In [6]:
import pickle 
load_file_name = f'/kaggle/input/linear_regression_nnm/pytorch/default/1/linear_regression_scale.pkl'

# 모델과 선택된 특성을 로드
with open(load_file_name, 'rb') as load_file:
    model_info = pickle.load(load_file)

standardscaler_feature = model_info['scale_feature']
standardscaler_label = model_info['scale_label']

print("모델과 선택된 특성이 로드되었습니다.")

모델과 선택된 특성이 로드되었습니다.


## 데이터 예측 서비스

In [7]:
import os
# 환경 변수에 따라 파일 경로 설정
if os.getenv('KAGGLE_KERNEL_RUN_TYPE') is not None:
    # Kaggle 환경
    train_file_path =  f'/kaggle/input/house-prices-advanced-regression-techniques/train.csv'
    test_file_path = f'/kaggle/input/house-prices-advanced-regression-techniques/test.csv'
    print("kaggle")
else:
    # Docker 환경
    train_file_path = "../../datasets/train.csv"
    test_file_path = "../../datasets/test.csv"
    print("another")
    

kaggle


In [8]:
import pandas as pd

train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [9]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [10]:
def get_continuous_columns(df, unique_threshold=10):
    """
    DataFrame에서 연속형 수치 변수를 찾는 함수
    
    Parameters:
    -----------
    df : pandas DataFrame
        분석할 데이터프레임
    unique_threshold : int, default=10
        연속형으로 간주할 최소 unique 값 개수
        
    Returns:
    --------
    list : 조건을 만족하는 컬럼명 리스트
    """
    
    continuous_columns = []
    
    for col in df.columns:
        # 1. object 타입이 아닌지 확인
        is_numeric = df[col].dtype != 'object'
        
        # 2. 결측치가 없는지 확인
        no_missing = df[col].isna().sum() == 0
        
        # 3. unique 값의 개수가 임계값보다 큰지 확인
        enough_unique = df[col].nunique() >= unique_threshold
        
        # 모든 조건을 만족하면 리스트에 추가
        if is_numeric and no_missing and enough_unique:
            print(df[col].nunique())
            continuous_columns.append(col)
    
    return continuous_columns

In [11]:
target_column = 'SalePrice'

continuous_cols = get_continuous_columns(train_df.drop(columns=['Id', target_column]), unique_threshold=50)
print("연속형 변수:", continuous_cols)

1073
112
61
637
144
780
721
753
417
861
441
274
202
120
76
연속형 변수: ['LotArea', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'ScreenPorch']


In [12]:
feature_columns = continuous_cols # ['BsmtUnfSF','LowQualFinSF','HalfBath','Fireplaces']

select_df = train_df[feature_columns]
select_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   LotArea        1460 non-null   int64
 1   YearBuilt      1460 non-null   int64
 2   YearRemodAdd   1460 non-null   int64
 3   BsmtFinSF1     1460 non-null   int64
 4   BsmtFinSF2     1460 non-null   int64
 5   BsmtUnfSF      1460 non-null   int64
 6   TotalBsmtSF    1460 non-null   int64
 7   1stFlrSF       1460 non-null   int64
 8   2ndFlrSF       1460 non-null   int64
 9   GrLivArea      1460 non-null   int64
 10  GarageArea     1460 non-null   int64
 11  WoodDeckSF     1460 non-null   int64
 12  OpenPorchSF    1460 non-null   int64
 13  EnclosedPorch  1460 non-null   int64
 14  ScreenPorch    1460 non-null   int64
dtypes: int64(15)
memory usage: 171.2 KB


In [13]:
y_train = train_df[target_column].values.reshape(-1, 1)  # 2차원 배열로 변환

y_train.shape

(1460, 1)

In [14]:
scailing_array_feature = standardscaler_feature.transform(select_df)
scailing_array_label = standardscaler_label.transform(y_train)
scailing_array_feature[5], scailing_array_label[5]

(array([ 0.3606161 ,  0.71978635,  0.49103992,  0.63244965, -0.28865283,
        -1.13928601, -0.59611547, -0.94869107,  0.50187469, -0.29214465,
         0.03284429, -0.43293146, -0.25153911, -0.3593249 , -0.27020835]),
 array([-0.47750496]))

In [15]:
train_features_tensor = torch.tensor(scailing_array_feature, dtype=torch.float32)
train_label_tensor = torch.tensor(scailing_array_label, dtype=torch.float32).view(-1,1) # 행만 있는 경우 행열로 변환 해줌
train_features_tensor.shape, train_label_tensor.shape

(torch.Size([1460, 15]), torch.Size([1460, 1]))

In [16]:
criterion = torch.nn.MSELoss() # Loss function 거의 label 타입에 따라 결정됨

In [17]:
with torch.no_grad(): # 학습 목정이 아닌 평가 목적 위해 고정
    pred_y = loaded_model(train_features_tensor)
    loss = criterion(pred_y, train_label_tensor) # 예측도 간은 loss function
    print(f"loss : {loss.item()}")

loss : 0.04772137105464935
